In [1]:
import sys
from os.path import join as pathjoin
from os import getcwd
sys.path.append(pathjoin('..', 'src'))
from ldmunit.models import decision_making, associative_learning
from ldmunit.models.utils import multi_from_single_interactive
from ldmunit.tests import NLLTest, BICTest
import sciunit
sciunit.settings['CWD'] = getcwd()
DATA_PATH = pathjoin('..', 'data')

from read_example_data import get_simulation_data, get_model_params

## Decision Making

In [3]:
MultiRWCKModel = multi_from_single_interactive(decision_making.RWCKModel)
MultiNWSLSModel = multi_from_single_interactive(decision_making.NWSLSModel)
MultiRandomRespondModel = multi_from_single_interactive(decision_making.RandomRespondModel)

In [5]:
rwck = get_simulation_data(pathjoin(DATA_PATH, 'multi-rwck.csv'), 3)
nll_rwck = NLLTest(name='RWCK sim', observation=rwck)

rr = get_simulation_data(pathjoin(DATA_PATH, 'multi-random-responding.csv'), 3)
nll_rr = NLLTest(name='RR sim', observation=rr)

nwsls = get_simulation_data(pathjoin(DATA_PATH,'multi-nwsls.csv'), 3)
nll_nwsls = NLLTest(name='NWSLS sim', observation=nwsls)

In [7]:
nll_suite = sciunit.TestSuite([nll_rwck, nll_rr, nll_nwsls], name="NLL suite")

In [11]:
n_action = 3
n_obs = 3

param_list = get_model_params(pathjoin(DATA_PATH, 'multi-rwck_prior.csv'))
multi_rwck = MultiRWCKModel(param_list, n_action=n_action, n_obs=n_obs)
multi_rwck.name = "rwck"

param_list = get_model_params(pathjoin(DATA_PATH, 'multi-random-responding_prior.csv'))
multi_rr = MultiRandomRespondModel(param_list, n_action=n_action, n_obs=n_obs)
multi_rr.name = "rr"

param_list = get_model_params(pathjoin(DATA_PATH,'multi-nwsls_prior.csv'))
multi_nwsls = MultiNWSLSModel(param_list, n_action=n_action, n_obs=n_obs)
multi_nwsls.name = 'nwsls'

In [12]:
nll_suite.judge([multi_rwck, multi_rr, multi_nwsls])

RWCK sim RR sim NWSLS sim
rwck       325    330       282
rr         381    301       372
nwsls      476    521       191

## Associative Learning

In [13]:
MultiKrwNormModel = multi_from_single_interactive(associative_learning.KrwNormModel)
MultiBetaBinomialModel = multi_from_single_interactive(associative_learning.BetaBinomialModel)
MultiLSSPDModel = multi_from_single_interactive(associative_learning.LSSPDModel)

In [15]:
krw_norm = get_simulation_data(pathjoin(DATA_PATH, 'multi-krw_norm.csv'), 3, True)
bic_krw_norm = BICTest(name="krw_norm sim", observation=krw_norm)

lsspd = get_simulation_data(pathjoin(DATA_PATH, 'multi-lsspd.csv'), 3, True)
bic_lsspd = BICTest(name='lsspd sim', observation=lsspd)

bb = get_simulation_data(pathjoin(DATA_PATH, 'multi-beta_binomial.csv'), 3, True)
bic_bb = BICTest(name='Beta Binomial sim', observation=bb)

In [17]:
bic_al_suite = sciunit.TestSuite([bic_krw_norm, bic_lsspd, bic_bb], name="BIC suite for associative learning")

In [18]:
param_list = get_model_params(pathjoin(DATA_PATH, 'multi-krw_norm_prior.csv'))
multi_krw_norm = MultiKrwNormModel(param_list, n_obs=4)
multi_krw_norm.name = "krw_norm"

param_list = get_model_params(pathjoin(DATA_PATH, 'multi-lsspd_prior.csv'))
multi_lsspd = MultiLSSPDModel(param_list, n_obs=4)
multi_lsspd.name = "lsspd"

param_list = get_model_params(pathjoin(DATA_PATH, 'multi-beta_binomial_prior.csv'))
multi_bb = MultiBetaBinomialModel(param_list, n_obs=4)
multi_bb.name = "bb"

In [19]:
bic_al_suite.judge([multi_krw_norm, multi_lsspd, multi_bb])

krw_norm sim lsspd sim Beta Binomial sim
krw_norm     2.37e+03  7.43e+16          3.86e+03
lsspd        2.62e+03   1.3e+16          2.77e+03
bb           2.12e+03     2e+16          2.09e+03